In [41]:
import numpy as np
import pandas as pd
import re

In [40]:
import os

folder_path = "../data/gaze/ROI" # sostituire "percorso_della_cartella" con il percorso della cartella che si vuole leggere

all_roi = []
for filename in os.listdir(folder_path):
    path_complete = os.path.join(folder_path, filename)
    if os.path.isfile(path_complete):
        all_roi.append(path_complete)

all_roi




['../data/gaze/ROI/roiLook052.asc.npy',
 '../data/gaze/ROI/roiLook042.asc.npy',
 '../data/gaze/ROI/roiLook030.asc.npy',
 '../data/gaze/ROI/roiLook020.asc.npy',
 '../data/gaze/ROI/roiLook029.asc.npy',
 '../data/gaze/ROI/roiLook016.asc.npy',
 '../data/gaze/ROI/roiLook006.asc.npy',
 '../data/gaze/ROI/roiLook028.asc.npy',
 '../data/gaze/ROI/roiLook007.asc.npy',
 '../data/gaze/ROI/roiLook017.asc.npy',
 '../data/gaze/ROI/roiLook043.asc.npy',
 '../data/gaze/ROI/roiLook053.asc.npy',
 '../data/gaze/ROI/roiLook021.asc.npy',
 '../data/gaze/ROI/roiLook031.asc.npy',
 '../data/gaze/ROI/roiLook048.asc.npy',
 '../data/gaze/ROI/roiLook015.asc.npy',
 '../data/gaze/ROI/roiLook005.asc.npy',
 '../data/gaze/ROI/roiLook033.asc.npy',
 '../data/gaze/ROI/roiLook023.asc.npy',
 '../data/gaze/ROI/roiLook051.asc.npy',
 '../data/gaze/ROI/roiLook041.asc.npy',
 '../data/gaze/ROI/roiLook022.asc.npy',
 '../data/gaze/ROI/roiLook032.asc.npy',
 '../data/gaze/ROI/roiLook050.asc.npy',
 '../data/gaze/ROI/roiLook049.asc.npy',


In [43]:
path1 = "../data/gaze/ROI/roiLook052.asc.npy"
path2 = "../data/gaze/ROI/roiLook002.asc.npy"

pattern = r"roiLook(\d+)\.asc\.npy"

match1 = re.search(pattern, path1)
if match1:
    print(int(match1.group(1)))  # Output: 052

match2 = re.search(pattern, path2)
if match2:
    print(int(match2.group(1)))  # Output: 002




52
2


In [53]:
def extract_sub(path):
    pattern = r"roiLook(\d+)\.asc\.npy"
    return int(re.search(pattern, path).group(1))

In [21]:
list_subject = [1]

In [47]:
df_ = pd.DataFrame(columns=['Subject', 'Trial', 'Fixation #', 'X_Coord', 'Y_Coord'])

for path in all_roi:
    data = np.load(path, allow_pickle=True)
    for i in range(len(data)):
        subject = extract_sub(path)
        trial = i+1
        fix_ = data[i]['fixations']
        for num, xy in enumerate(fix_):
            coords = tuple(np.round(xy['mean'], 2))
            row = [int(subject), int(trial), int(num+1), coords[0], coords[1]]
            df_.loc[len(df_)] = row

In [48]:
len(df_)

222864

In [50]:
df_.to_csv('../data/gaze/tmp/tmp_fixations.csv', index=False)

In [61]:
baseline = pd.read_csv('../data/gaze/baselines/10.csv')
baseline

,RECORDING_SESSION_LABEL,TRIAL_LABEL,CURRENT_FIX_X,CURRENT_FIX_Y,IS_VALID
0,10,1,946.200000,505.500000,True
1,10,2,941.400000,502.700000,True
2,10,3,954.900000,500.500000,True
3,10,4,946.700000,503.700000,True
4,10,5,934.500000,498.700000,True
...,...,...,...,...,...
154,10,156,937.100000,496.800000,True
155,10,157,937.600000,502.100000,True
156,10,158,931.400000,503.700000,True
157,10,159,961.412121,571.755556,True


In [51]:
df_updated = df_.copy(deep=True)
x_cross = 1920/2
y_cross = 1080/2



In [55]:
for path in all_roi:
    subj = extract_sub(path)
    print(pd.read_csv('../data/gaze/baselines/'+str(subj)+'.csv'))
    break

     RECORDING_SESSION_LABEL  TRIAL_LABEL  CURRENT_FIX_X  CURRENT_FIX_Y  \
0                         52            1     942.000000     528.500000   
1                         52            2     948.800000     526.700000   
2                         52            3     938.426165     530.982079   
3                         52            4     940.200000     525.400000   
4                         52            5     937.300000     524.700000   
..                       ...          ...            ...            ...   
154                       52          156     966.400000     521.800000   
155                       52          157    1002.202899     499.201449   
156                       52          158     941.300000     501.200000   
157                       52          159     961.700000     501.300000   
158                       52          160     951.081250     510.293750   

     IS_VALID  
0        True  
1        True  
2        True  
3        True  
4        True  
.. 

In [38]:

for i in range(len(baseline)):
    trial = baseline.TRIAL_LABEL[i]
    valid = baseline.IS_VALID[i]

    if valid:
        x_base = baseline.CURRENT_FIX_X[i]
        y_base = baseline.CURRENT_FIX_Y[i]

    else:
        x_base = baseline[baseline['IS_VALID']==True]['CURRENT_FIX_X'].mean()
        y_base = baseline[baseline['IS_VALID']==True]['CURRENT_FIX_Y'].mean()

    delta_x, delta_y = (x_cross-x_base), (y_cross-y_base)

    for row in df_[df_['Trial'] == trial].iterrows():
        df_updated.loc[row[0], 'X_Coord'] = np.round(row[1]['X_Coord'] + delta_x,2)
        df_updated.loc[row[0], 'Y_Coord'] = np.round(row[1]['Y_Coord'] + delta_y, 2)



In [39]:
df_updated

,Subject,Trial,Fixation #,X_Coord,Y_Coord
0,1.0,1.0,1.0,953.20,542.09
1,1.0,1.0,2.0,951.44,538.50
2,1.0,1.0,3.0,1222.36,851.90
3,1.0,1.0,4.0,968.43,535.66
4,1.0,1.0,5.0,953.13,544.87
...,...,...,...,...,...
5366,1.0,160.0,31.0,959.89,631.15
5367,1.0,160.0,32.0,963.91,640.30
5368,1.0,160.0,33.0,966.12,640.28
5369,1.0,160.0,34.0,903.96,519.44
